In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-11-13 03:07:00--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-11-13 03:07:00 (10.6 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Jewelry_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

### Create DataFrames to match vine_table

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R135Q3VZ4DQN5N|          5|            0|          0|   N|                Y|
|R2N0QQ6R4T7YRY|          5|            0|          0|   N|                N|
|R3N5JE5Y4T6W5M|          5|            0|          0|   N|                Y|
|R2I150CX5IVY9Q|          5|            0|          0|   N|                Y|
|R1RM9ICOOA9MQ3|          5|            0|          0|   N|                Y|
|R2J2KMDL10UMSH|          5|            0|          0|   N|                Y|
|R3R9ZUFA4TB4FQ|          5|            0|          0|   N|                Y|
|R3UQ8VAQN7R6WL|          5|            0|          0|   N|                Y|
|R1FXZ69C01JNQM|          5|            0|          0|   N|                Y|
| RY36LB4OW0FFS|          5|            0|          0|   N|     

In [7]:
total_votes_greater_20_df = vine_df.filter("total_votes >= 20")
total_votes_greater_20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|
| R36LII9IITE17|          1|           22|         24|   N|                Y|
|R35VKNE16PFY0H|          5|           22|         24|   N|                N|
|R3TG008LHHZZIN|          5|          123|        125|   N|                Y|
|R2FYQBKCC1XG4M|          5|           20|         20|   N|                Y|
| R63PV336NI5X7|          1|           27|         28|   N|                Y|
|R1R42WPHB5ZSWI|          5|           51|         52|   N|     

In [8]:
helpful_votes_greater_50_df = total_votes_greater_20_df.filter("helpful_votes / total_votes >= 0.5")
helpful_votes_greater_50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|
| R36LII9IITE17|          1|           22|         24|   N|                Y|
|R35VKNE16PFY0H|          5|           22|         24|   N|                N|
|R3TG008LHHZZIN|          5|          123|        125|   N|                Y|
|R2FYQBKCC1XG4M|          5|           20|         20|   N|                Y|
| R63PV336NI5X7|          1|           27|         28|   N|                Y|
|R1R42WPHB5ZSWI|          5|           51|         52|   N|     

In [9]:
vine_review_df = helpful_votes_greater_50_df.filter("vine == 'Y'")
vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R15PXA9XU9ZKSU|          4|           29|         29|   Y|                N|
|R2LJ4QYQ4PYDEH|          5|           24|         26|   Y|                N|
|R2PJWWQZ37WCIC|          5|           95|         96|   Y|                N|
|R33G2VDKRN2USY|          5|           21|         21|   Y|                N|
|R13R4X37T2U50A|          4|           52|         57|   Y|                N|
|R1X1J2NAQ39DF6|          4|           29|         34|   Y|                N|
|R2UV5XHFNMPXOX|          5|           60|         61|   Y|                N|
|R31V08BCC9X3MF|          5|           18|         22|   Y|                N|
|R2G00QWTYW9N2G|          5|           86|         91|   Y|                N|
| RL8H3NKKP989J|          5|           30|         34|   Y|     

In [10]:
non_vine_review_df = helpful_votes_greater_50_df.filter("vine == 'N'")
non_vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|
| R36LII9IITE17|          1|           22|         24|   N|                Y|
|R35VKNE16PFY0H|          5|           22|         24|   N|                N|
|R3TG008LHHZZIN|          5|          123|        125|   N|                Y|
|R2FYQBKCC1XG4M|          5|           20|         20|   N|                Y|
| R63PV336NI5X7|          1|           27|         28|   N|                Y|
|R1R42WPHB5ZSWI|          5|           51|         52|   N|     

In [15]:
def review_summary(df):
  total_reviews = df.count()
  count_5_stars = df.filter("star_rating == 5").count()
  percent_5 = round((float(count_5_stars)/float(total_reviews)) * 100,2)
  rv = {
      "Total Reviews": total_reviews,
      "Total 5-Star Reviews": count_5_stars,
      "5 Star Review Percentage": percent_5,
  }
  return rv


In [16]:
review_summary(vine_review_df)


{'Total Reviews': 21,
 'Total 5-Star Reviews': 11,
 '5 Star Review Percentage': 52.38}

In [17]:
review_summary(non_vine_review_df)

{'Total Reviews': 7689,
 'Total 5-Star Reviews': 4444,
 '5 Star Review Percentage': 57.8}